![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [58]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [59]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

## Getting Started
I first ran the `.info` method on `ds_jobs_transformed` to see what dtype all the columns are. I then created a function that printed out how many unique values each column had so I could get a better idea of what data I had in each column. 

In [60]:
print(ds_jobs_transformed.info())
for col in ds_jobs_transformed:
    print ('# of unique values in '+ col + ' :' + str(len(ds_jobs_transformed[col].value_counts())))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

The first requirement is to set columns containing categories with only two factors as Booleans. From the cell above, I see the two columns that need to be stored as Booleans are the 'relevant_experience' and 'job_change' columns. I printed out the unique values of each column to see what I was working with in each column. As the data in the 'relevant_experience' column is of the dtype `object`, I first converted the data to the dtype `category`, then used Boolean indexing to assign 'Has relevant experience' to `True`, and 'No relevant experience' to `False`.
As the data in the 'job_change' column is of the dtype `float64` and is already stored as 1 and 0, all I had to do was run the method `.astype('bool')` to convert them to booleans.

In [61]:
print(ds_jobs_transformed['relevant_experience'].unique())
print(ds_jobs_transformed['job_change'].unique())

['Has relevant experience' 'No relevant experience']
[1. 0.]


In [62]:
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype('category')
ds_jobs_transformed['relevant_experience'] = np.where(
    ds_jobs_transformed['relevant_experience'].str.contains('Has', regex=False), True, False)
print(ds_jobs_transformed['relevant_experience'].unique())

[ True False]


In [63]:
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('bool')
print(ds_jobs_transformed['job_change'].dtype)
print(ds_jobs_transformed['job_change'].unique())

bool
[ True False]


The next requirement was to store columns containing integers only as 32-bit integers. Looking back at the data, I see that there are a few columns with the dtype `int64`. I used a for loop to identify these columns, saving the column names in a list called `int_only_columns`, and then changed their dtypes to `int32`. 

In [64]:
int_only_columns = []
for col in ds_jobs_transformed:
    if ds_jobs_transformed[col].dtype == 'int64':
        int_only_columns.append(col)
ds_jobs_transformed[int_only_columns] = ds_jobs_transformed[int_only_columns].astype('int32')
print(ds_jobs_transformed.dtypes)

student_id                  int32
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object


To change the columns containing floats to 16 bit floats, I did used the same method I used for converting the integer columns to `int32`.

In [65]:
float_only_columns = []
for col in ds_jobs_transformed:
    if ds_jobs_transformed[col].dtype == 'float64':
        float_only_columns.append(col)
ds_jobs_transformed[float_only_columns] = ds_jobs_transformed[float_only_columns].astype('float16')
print(ds_jobs_transformed.dtypes)

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object


For the third and forth requirements of storing categorical data as the `category` type, I used a for loop to assign all columns containing the dtype `object` to `column`. This was all I had to do for the nominal categorical data, however the ordinal categorical data required more work.  

In [66]:
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,True
1,29725,city_40,0.775879,Male,False,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,False
2,11561,city_21,0.624023,NaN,False,Full time course,Graduate,STEM,5,NaN,NaN,never,83,False
3,33241,city_115,0.789062,NaN,False,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,True
4,666,city_162,0.767090,Male,True,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,False


In [67]:
cat_columns = []
for col in ds_jobs_transformed:
    if ds_jobs_transformed[col].dtype == 'object':
        cat_columns.append(col)
print(cat_columns)
ds_jobs_transformed[cat_columns] = ds_jobs_transformed[cat_columns].astype('category')
print(ds_jobs_transformed.dtypes)
#nom_cat_columns = ['gender', 'major_discipline','company_type']
#ds_jobs_transformed[nom_cat_columns] = ds_jobs_transformed[nom_cat_columns].astype('category')
#print(ds_jobs_transformed.dtypes)


['city', 'gender', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']
student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object


The first column to store the data as an ordered category was the 'city' column, and I'm pretty sure I did so in the least efficient way possible. I used a bunch of for loops and new lists to create the list `list_of_cities_ordered` that listed all the different categories in order. I then passed this list into `.cat.reorder_categories` to reorder the categories in the `city` column. 

In [68]:
list_of_cities = list(ds_jobs_transformed['city'].cat.categories)
print(list_of_cities)
list_of_cities_num_str = []
for city in list_of_cities:
    list_of_cities_num_str.append(city.replace('city_', ''))

list_of_cities_num = []
for num in list_of_cities_num_str:
    list_of_cities_num.append(int(num))
#print(list_of_cities_num)
list_of_cities_num.sort()
#print(list_of_cities_num)

list_of_cities_str_num = []
for num in list_of_cities_num:
    list_of_cities_str_num.append(str(num))
#print(list_of_cities_str_num)

list_of_cities_ordered = []
for city in list_of_cities_str_num:
    list_of_cities_ordered.append('city_' + city)
print(list_of_cities_ordered)


['city_1', 'city_10', 'city_100', 'city_101', 'city_102', 'city_103', 'city_104', 'city_105', 'city_106', 'city_107', 'city_109', 'city_11', 'city_111', 'city_114', 'city_115', 'city_116', 'city_117', 'city_118', 'city_12', 'city_120', 'city_121', 'city_123', 'city_126', 'city_127', 'city_128', 'city_129', 'city_13', 'city_131', 'city_133', 'city_134', 'city_136', 'city_138', 'city_139', 'city_14', 'city_140', 'city_141', 'city_142', 'city_143', 'city_144', 'city_145', 'city_146', 'city_149', 'city_150', 'city_152', 'city_155', 'city_157', 'city_158', 'city_159', 'city_16', 'city_160', 'city_162', 'city_165', 'city_166', 'city_167', 'city_171', 'city_173', 'city_175', 'city_176', 'city_179', 'city_18', 'city_180', 'city_19', 'city_2', 'city_20', 'city_21', 'city_23', 'city_24', 'city_25', 'city_26', 'city_27', 'city_28', 'city_30', 'city_31', 'city_33', 'city_36', 'city_37', 'city_39', 'city_40', 'city_41', 'city_42', 'city_43', 'city_44', 'city_45', 'city_46', 'city_48', 'city_50', 'c

In [69]:
print(ds_jobs_transformed.dtypes)
ds_jobs_transformed['city'] = ds_jobs_transformed['city'].cat.reorder_categories(new_categories = list_of_cities_ordered, ordered=True)
print(ds_jobs_transformed['city'].dtype)
print(ds_jobs_transformed['city'].cat.categories)

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object
category
Index(['city_1', 'city_2', 'city_7', 'city_8', 'city_9', 'city_10', 'city_11',
       'city_12', 'city_13', 'city_14',
       ...
       'city_162', 'city_165', 'city_166', 'city_167', 'city_171', 'city_173',
       'city_175', 'city_176', 'city_179', 'city_180'],
      dtype='object', length=123)


For the remaining columns containing ordinal categorical data ('enrolled_university', 'education_level', 'experience', 'company_size', 'last_new_job') the process of reordering them was pretty straightforward. Since they didn't have too many categories, I printed out the list of categories each column had, then manually entered them in the correct order in `.cat.reorder_categories()` for each column. 

In [70]:
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,True
1,29725,city_40,0.775879,Male,False,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,False
2,11561,city_21,0.624023,NaN,False,Full time course,Graduate,STEM,5,NaN,NaN,never,83,False
3,33241,city_115,0.789062,NaN,False,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,True
4,666,city_162,0.767090,Male,True,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,False


In [71]:
print(ds_jobs_transformed['enrolled_university'].cat.categories)
print(ds_jobs_transformed['education_level'].cat.categories)
print(ds_jobs_transformed['experience'].cat.categories)
print(ds_jobs_transformed['company_size'].cat.categories)
print(ds_jobs_transformed['last_new_job'].cat.categories)

Index(['Full time course', 'Part time course', 'no_enrollment'], dtype='object')
Index(['Graduate', 'High School', 'Masters', 'Phd', 'Primary School'], dtype='object')
Index(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '3', '4', '5', '6', '7', '8', '9', '<1', '>20'],
      dtype='object')
Index(['10-49', '100-499', '1000-4999', '10000+', '50-99', '500-999',
       '5000-9999', '<10'],
      dtype='object')
Index(['1', '2', '3', '4', '>4', 'never'], dtype='object')


In [72]:
ds_jobs_transformed['enrolled_university']= ds_jobs_transformed['enrolled_university'].cat.reorder_categories(
    new_categories = ['no_enrollment', 'Part time course', 'Full time course'], ordered=True)

ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].cat.reorder_categories(
    new_categories = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)

ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].cat.reorder_categories(
    new_categories = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'], ordered=True)

ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].cat.reorder_categories(
    new_categories = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'], ordered=True)

ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.reorder_categories(
    new_categories=['never', '1', '2', '3','4', '>4'], ordered=True )

In [73]:
print(ds_jobs_transformed['enrolled_university'].cat.categories)
print(ds_jobs_transformed['education_level'].cat.categories)
print(ds_jobs_transformed['experience'].cat.categories)
print(ds_jobs_transformed['company_size'].cat.categories)
print(ds_jobs_transformed['last_new_job'].cat.categories)

Index(['no_enrollment', 'Part time course', 'Full time course'], dtype='object')
Index(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], dtype='object')
Index(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '>20'],
      dtype='object')
Index(['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999',
       '10000+'],
      dtype='object')
Index(['never', '1', '2', '3', '4', '>4'], dtype='object')


For the final task of filtering the dataframe to only contain students with 10 or more years of experience and companies with at least 1000 employees, I simply used indexing to filter the results and saved it as a new version of `ds_jobs_transformed`. I then printed out the info for the dataset, which shows that the memory usage decreased down to 69.5KB, whereas it started at over 2.0 MB. 

In [74]:
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & \
                                        (ds_jobs_transformed['company_size'] >= '1000-4999')]

ds_jobs_transformed.head()
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)